# Huggingface Token setzen

In [1]:
import os
hf_token = "hf_Token"
#hf_token = os.getenv("HF_TOKEN")

# Git Repo per HTTPs Clonen

In [ ]:
!git clone https://github.com/qvest-digital/Workshop_Agentic_AI.git

# Pfad setzen

In [2]:
#SYSTEM_PATH = "/home/simon/Workshop_Agentic_AI"
SYSTEM_PATH = "./Workshop_Agentic_AI"

# Requirements installieren

In [ ]:
!pip install -r "$SYSTEM_PATH/requirements.txt"

# Speicherfragmentierung minimieren

In [3]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Workshop Agentic AI

Dieses Notebook zeigt Schritt für Schritt, wie ein agentisches KI-System aufgebaut wird, das RAG, externe Tool-Abfragen und LLM-Reasoning kombiniert. Ziel ist es, Halluzinationen zu reduzieren, Fakten deterministisch zu beziehen und mehrere spezialisierte Sub-Modelle zu orchestrieren.

## Ablauf
- Teil 1: Lokales LLM + Halluzinationen im Griff behalten
    - Lokales LLM laden (HF-Pipeline)
    - Sampling-Verhalten & Halluzinationen untersuchen
    - Textgenerierung konfigurieren (Determinismus vs. Kreativität)
- Teil 2: Retrieval-Augmented Generation (RAG) mit dem Fantasietier Razepato
    - Texte als Embeddings in eine Vektordatenbank schreiben und vergleichen
    - Retrieval + Query + Antwortgenerierung
    - Was RAG im kontext LLM leistet
    - RAG und LLM verbinden
- Teil 3: Erlangen von Wissen durch Nutzung von MCP-Tools
    - MCP-Tools anbinden (manuell)
    - Fakten über MCP-Aufrufe beziehen (z. B. Koordinaten, Wetter, Flächen)
- Teil 4: Voll agentisches MCP – der Mathematikassistent denkt und handelt selbst
    - Planen & Reasoning (Chain-of-Thought, ReAct)
    - Sub-LLMs orchestrieren (Planner → Validator → Executor → Auditoren → Renderer)

# Timetable

| Zeit        | Thema                          | Inhalt / Ziel                                                |
|-------------|--------------------------------|--------------------------------------------------------------|
| 09:00–09:30 | Einchecken & Agenda vorstellen | Tagesübersicht                                               |
| 09:30–10:00 | LLM + Parameter                | HF-Setup, Tuning-Parameter, Halluzinationen => (Experimente) |
| 10:00–10:30 | Prompting (Basisprompt)        | Prompt-Engeneering, Experimente, Diskussion => (Experimente) |
| 10:30–12:00 | RAG                            | Embeddings, Vektorsuche, Retrieval, Integration              |
| 12:00–13:00 | Mittag                         | Pause                                                        |
| 13:00–13:20 | MCP-Basics                     | Konzept, Server, Call-Mechanik                               |
| 13:20–14:00 | MCP-Tool-Use                   | Fakten (Koordinaten, Wetter, Locations) über Tools           |
| 14:00–15:30 | MCP-Beispiel (manuel)          | Grundfuntkionen von MCP verstehen                            |
| 15:30–16:00 | Übergang zu Agenten            | Wie aus RAG + MCP → agentisches System wird                  |
| 16:00–16:30 | Code-Walkthrough               | Pipeline-Durchgang, Q&A                                      |
| 16:30       | Ende                           | Abschluss                                                    |

Metakommentar:
- vor jedem Block Reflexionsfragen zu Verknüpfung & Möglichkeiten.
- nach jedem Block Verbesserungsideen.

# Herstellen der Vorbedingungen aus Teil 1:


## Modell laden: lokal oder von Hugging Face

### Funktion
- Lädt ein Llama-3.1-Instruct-Modell entweder:
  - lokal aus dem Ordner ./models/llama-3.1-8b, wenn der Pfad vorhanden ist, oder
  - online von Hugging Face mit der Modell-ID meta-llama/Llama-3.1-8B-Instruct, und speichert es anschließend lokal ab.
- Die Umgebungsvariablen werden mit load_dotenv() aus einer .env-Datei geladen, u. a. das Hugging-Face-Token.

### Inputs
- Dateisystem:
  - Existenz von MODEL_PATH (./models/llama-3.1-8b).
- Umgebungsvariable:
  - HF_TOKEN (wird mit os.getenv("HF_TOKEN") gelesen) – persönliches Zugriffstoken für Hugging Face.
- Hyperparameter:
  - MODEL_ID gibt die zu ladende Modell-ID an.
- Hardware:
  - device_map="auto" versucht automatisch, GPU(s) oder CPU sinnvoll zu nutzen.
  - torch_dtype="auto" bzw. dtype="auto" lässt das Modell selbst einen sinnvollen Datentyp wählen (z. B. bfloat16 oder float16).

### Outputs
- Globale Python-Variablen:
  - tokenizer: Instanz von AutoTokenizer, konfiguriert für das Llama-3.1-Modell.
  - model: Instanz von AutoModelForCausalLM, bereit für Textgenerierung.

In [4]:
from dotenv import load_dotenv
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

load_dotenv()

MODEL_PATH = f"{SYSTEM_PATH}/models/llama-3.1-8b"
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

if os.path.exists(MODEL_PATH):
    print("Lade Modell lokal …")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained( MODEL_PATH, torch_dtype="auto", device_map="auto" )

else:
    print("Lade Modell von Hugging Face …")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=hf_token, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID, token=hf_token, dtype="auto", device_map="auto")

    # lokal speichern
    model.save_pretrained(MODEL_PATH)
    tokenizer.save_pretrained(MODEL_PATH)



Lade Modell lokal …


The tokenizer you are loading from '/home/simon/Workshop_Agentic_AI/models/llama-3.1-8b' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Pipeline erstellen
### Funktion
- Baut eine Hugging-Face-pipeline für Textgenerierung auf Basis des zuvor geladenen Modells und Tokenizers.
- Diese Pipeline kapselt:
  - Tokenisierung,
  - das Aufrufen des Modells,
  - und das Zurückkonvertieren der Token in Text.

### Inputs
- model: Causal-Language-Model (AutoModelForCausalLM), im vorherigen Block geladen.
- tokenizer: passender Tokenizer zu diesem Modell (AutoTokenizer).
- Task-Typ: "text-generation" – legt fest, dass es sich um eine generative Textaufgabe handelt.

### Outputs
- Variable:
  - llm: eine aufrufbare Pipeline-Instanz.

### Rückgabewert bei Aufruf von llm(...):
```python
[
  {
    "generated_text": "Vollständiger generierter Text (inkl. Prompt oder abhängig von den Parametern)"
  }
]
```

In [5]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


## Einfache Chat-Funktion zur Verfügung stellen

### Funktion
- Stellt ein vereinfachtes Chat-Interface zur Verfügung, das direkt mit einem String arbeitet.
- Die Funktion:
  - ruft intern die llm-Pipeline auf,
  - übergibt alle relevanten Generationsparameter,
  - und gibt am Ende nur den reinen generierten Text zurück (str statt verschachtelte Struktur).

### Inputs
- Pflichtparameter:
  - prompt: str – der Eingabetext an das Modell.
- Optionale Parameter zum Experimentieren
  - max_new_tokens: maximale Anzahl neu zu generierender Token.
  - temperature: steuert die Zufälligkeit (0 ≈ deterministischer, >0 zufälliger).
  - top_k: Sampling nur aus den k wahrscheinlichsten Token.
  - top_p: „Nucleus Sampling“ – Auswahl aus der kleinsten Masse der wahrscheinlichsten Token, deren Summe ≥ p ist.
  - repetition_penalty: >1.0 bestraft Wiederholungen.
  - num_beams, num_beam_groups, diversity_penalty: Parameter für Beam Search (systematisches Durchsuchen mehrerer Kandidaten).
  - early_stopping: beendet Beam Search frühzeitig, wenn bestimmte Kriterien erfüllt sind.

### Outputs
- Rückgabewert:
  - str: der vom Modell generierte Antworttext (out[0]["generated_text"] ohne führende/trailing Leerzeichen).

### Typische Verwendung:
- llama_chat("Erkläre mir kurz, was ein LLM ist.")

In späteren Zellen wird statt eines rohen Prompts ein Chat-Prompt übergeben, der mit build_chat_prompt erzeugt wird.

In [6]:
def llama_chat(
        prompt: str,
        max_new_tokens: int = 128,
        temperature: float = 0.01,
        top_k: int = 50,
        top_p: float = 1.0,
        typical_p: float = 1.0,
        repetition_penalty: float = 1.0,
        length_penalty: float = 1.0,
        no_repeat_ngram_size: int = 0,
        num_beams: int = 1,
        num_beam_groups: int = 1,
        diversity_penalty: float = 0.0,
        early_stopping: bool = False,) -> str:
    """Sehr simples Wrapper-Interface.
    Wir verwenden ein 'single prompt' Format, um es notebook-tauglich zu halten.
    """
    out = llm(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        typical_p=typical_p,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams,
        num_beam_groups=num_beam_groups,
        diversity_penalty=diversity_penalty,
        early_stopping=early_stopping,
        return_full_text=False,
        eos_token_id=llm.tokenizer.eos_token_id,
        pad_token_id=llm.tokenizer.pad_token_id,
    )
    return out[0]["generated_text"].strip()

Zeit zum Experimentieren (5 - 10 min.)

# Teil 2: Retrieval-Augmented Generation (RAG) mit dem erfundenen Mathematiker Maximus Qvestus der III

In diesem zweiten Teil bauen wir um unser lokales LLM herum ein kleines RAG-System:
1. Der erfundene Mathematiker Mathematiker Maximus Qvestus der III liegt als Textdatei auf der Festplatte.
2. Der Text wird:
    - eingelesen,
    - in Chunks zerlegt,
    - mit einem Embedding-Modell in Vektoren kodiert,
    - in einem FAISS-Index gespeichert.
3. Im Anschluss utzen wir diese Vektorembeddings um relevante Textstellen wiederfinden. Zuerst machen wir das ohne LLM.
4. Danach erweitern wir den System-Prompt um die RAG-Logik und bereiten das LLM auf den neuen Context vor.
5. Schließlich nutzen wir diesen System um das LLM Fragen über unser Fantasietier beantworten zu lassen, indem es den retrieveten Kontext zum Razepato einbezieht.

##  Maximus Qvestus der III-Datei einlesen

### Funktion
- Liest den vollständigen Inhalt der Datei RAG_Data/Maximus_Qvestus_III.txt ein.
- Gibt einen Ausschnitt der ersten Zeichen aus, um grob zu prüfen, ob der Inhalt stimmt (Sanity Check).

### Input
- Dateipfad: RAG_Data/Maximus_Qvestus_III.txt
- encoding der Datei.

### Output
- Variable full_text: str – enthält den kompletten Maximus_Qvestus_III-Text.
- Konsolenausgabe der ersten ~800 Zeichen (print(full_text[:800])) zur visuellen Kontrolle, dass:
    - die Datei gefunden wurde,
    - das Encoding stimmt,
    - und tatsächlich der erwartete Inhalt geladen wurde.

In [7]:
# Datei einlesen – Pfad ggf. anpassen
with open(f"{SYSTEM_PATH}/RAG_Data/Maximus_Qvestus_III.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

print(full_text[:800])  # kurz prüfen

Maximus Qvestus der III. wurde angeblich in einer Nacht geboren, in der ein seltener Komet genau über der kleinen Universitätsstadt Valebris stand. Seine Eltern waren Buchbinder, und so wuchs er zwischen leise raschelnden Seiten auf, lange bevor er lesen konnte. Mit sechs Jahren soll er begonnen haben, die Pflastersteine der Straße nach symmetrischen Mustern zu zählen — nicht aus Zwang, sondern aus reiner Freude an der Ordnung, die sich unter dem scheinbaren Chaos verbarg.

Sein größtes Werk, die sogenannte „Theorie der wandernden Zahlen“, entstand aus einer einfachen Beobachtung: Zahlen, so meinte Qvestus, verhalten sich wie Reisende — sie ändern ihre Bedeutung je nachdem, in welchem System man sie betrachtet. Kollegen hielten ihn zunächst für exzentrisch, doch seine Vorlesungen waren leg


## Sätze mit NLTK tokenisieren

### Funktion
- Installiert die notwendigen NLTK-Ressourcen und zerlegt den eingelesenen Text full_text in einzelne Sätze.
- Satzgrenzen sind später wichtig, um in sinnvolle Chunks zu schneiden (nicht mitten im Satz abbrechen).

### Input
- full_text: str aus dem vorherigen Schritt.
- NLTK-Funktionen:
    - nltk.download("punkt")
    - nltk.download("punkt_tab") (aktuelle NLTK-Struktur)
    - sent_tokenize(full_text, language="german")

### Output
- Variable sentences: List[str] – Liste aller erkannten Sätze im Text.

### Weiteres:
- NLTK lädt die Punkt-Modelle lokal (einmalig), damit sent_tokenize für Deutsch funktioniert.

In [8]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
nltk.download("punkt_tab")

sentences = sent_tokenize(full_text, language="german")

[nltk_data] Downloading package punkt to /home/simon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/simon/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Text in überlappende Chunks zerlegen

### Funktion
- Zerlegt den Maximus_Qvestus_III-Text in semantisch sinnvolle Textblöcke (Chunks), die:
    - nicht zu lang sind (MAX_CHARS),
    - Satzgrenzen respektieren,
    - optional einen Overlap haben, damit Informationen an Chunk-Grenzen nicht verloren gehen.
- Das verbessert die Qualität der späteren Retrieval-Ergebnisse.

### Input
- full_text: str – Rohtext.
- Hyperparameter:
    - MAX_CHARS = 100 – maximale Zeichenlänge pro Chunk.
    - OVERLAP_SENTENCES = 1 – Anzahl der Sätze, die von einem Chunk in den nächsten „überlappen“.

### Logik:
- raw_paragraphs: Aufteilung nach Doppel-Newlines (\n\n) → Absatzliste.
- Für jeden Absatz:
    - Wenn kurz genug: direkt als Chunk übernehmen.
    - Wenn zu lang: in Sätze splittet (sent_tokenize) und iterativ Chunks bis MAX_CHARS aufbauen.
    - Beim Chunk-Wechsel werden die letzten OVERLAP_SENTENCES Sätze in den neuen Chunk übernommen.

### Output
- chunks: List[str] – Liste von Textblöcken, die der „Dokumentkorpus“ für das RAG werden.

### Konsolenausgabe:
- Anzahl der Chunks: print(f"Anzahl der Chunks: {len(chunks)}")
- Vorschau auf alle Chunks (Chunk 1, Chunk 2, …) zur manuellen Kontrolle:
    - Sind sie lesbar?
    - Schneiden sie nicht mitten in Wörtern/Sätzen?
    - Ist der Overlap sinnvoll?

In [16]:
from nltk.tokenize import sent_tokenize  # oder eigene Sentence-Split-Logik

MAX_CHARS = 50
OVERLAP_SENTENCES = 0  # z.B. 1 Satz Overlap

raw_paragraphs = [p.strip() for p in full_text.split("\n\n") if p.strip()]
chunks = []

for para in raw_paragraphs:
    # wenn der Absatz kurz ist, einfach direkt übernehmen
    if len(para) <= MAX_CHARS:
        chunks.append(para)
        continue

    # sonst: in Sätze splitten und mit Overlap chunken
    sentences = sent_tokenize(para)

    buffer_sents = []
    buffer_len = 0

    for sentence in sentences:
        sentence = sentence.strip()
        sentence_len = len(sentence) + 1  # +1 für Leerzeichen

        if buffer_len + sentence_len <= MAX_CHARS or not buffer_sents:
            # Satz passt noch in den aktuellen Chunk
            buffer_sents.append(sentence)
            buffer_len += sentence_len
        else:
            # aktueller Chunk ist voll → Chunk abschließen
            chunks.append(" ".join(buffer_sents))

            # Overlap: die letzten N Sätze in den neuen Chunk übernehmen
            if OVERLAP_SENTENCES > 0:
                overlap = buffer_sents[-OVERLAP_SENTENCES:]
            else:
                overlap = []

            buffer_sents = overlap + [sentence]
            buffer_len = sum(len(s) + 1 for s in buffer_sents)

    # was im Buffer übrig ist, auch noch als Chunk speichern
    if buffer_sents:
        chunks.append(" ".join(buffer_sents))

print(f"Anzahl der Chunks: {len(chunks)}\n")
for i, chunk in enumerate(chunks, start=1):
    print(f"--- Chunk {i} ---\n{chunk}\n")

Anzahl der Chunks: 8

--- Chunk 1 ---
Maximus Qvestus der III.

--- Chunk 2 ---
wurde angeblich in einer Nacht geboren, in der ein seltener Komet genau über der kleinen Universitätsstadt Valebris stand.

--- Chunk 3 ---
Seine Eltern waren Buchbinder, und so wuchs er zwischen leise raschelnden Seiten auf, lange bevor er lesen konnte.

--- Chunk 4 ---
Mit sechs Jahren soll er begonnen haben, die Pflastersteine der Straße nach symmetrischen Mustern zu zählen — nicht aus Zwang, sondern aus reiner Freude an der Ordnung, die sich unter dem scheinbaren Chaos verbarg.

--- Chunk 5 ---
Sein größtes Werk, die sogenannte „Theorie der wandernden Zahlen“, entstand aus einer einfachen Beobachtung: Zahlen, so meinte Qvestus, verhalten sich wie Reisende — sie ändern ihre Bedeutung je nachdem, in welchem System man sie betrachtet.

--- Chunk 6 ---
Kollegen hielten ihn zunächst für exzentrisch, doch seine Vorlesungen waren legendär; er zeichnete Gleichungen mit farbiger Kreide und behauptete, jede Farbe

## Embeddings mit SentenceTransformers + FAISS-Index

### Funktion
- Kodiert jeden Chunk in einen dichten Vektor (Embedding) und legt diese Vektoren in einem FAISS-Index ab.
- Das ist unser „Wissensspeicher“: statt mit Strings suchen wird später im Vektorraum.

### Input
- corpus = chunks – Liste der Chunk-Strings.
- Embedding-Modell:
    - EMBED_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    - Dieses Modell ist mehrsprachig und deckt Deutsch gut ab.

### Output
- embed_model: trainiertes Sentence-Transformers-Modell zum Kodieren neuer Anfragen.
- embeddings: np.ndarray – Matrix der Größe (num_chunks, dim).
- index: faiss.IndexFlatL2 – FAISS-Index für L2-Distanz-Suche.

### Konsolenausgabe:
- "Anzahl Vektoren im Index:", gefolgt von der Anzahl der Chunks (index.ntotal).

In [17]:
from sentence_transformers import SentenceTransformer
import faiss

# Multilinguales Embedding-Modell (Deutsch gut abgedeckt)
EMBED_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

# Korpus: ein Dokument = kompletter Inhalt von Razepato.txt
corpus = chunks

# Embedding berechnen
embeddings = embed_model.encode(corpus, convert_to_numpy=True, batch_size=32, show_progress_bar=True)

# FAISS Index anlegen
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("Anzahl Vektoren im Index:", index.ntotal)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Anzahl Vektoren im Index: 8


## RAG-Kontext aus dem Index abrufen: get_rag_context

### Funktion
1. Codiert eine natürliche Sprachfrage (prompt) in einen Vektor
2. Sucht im FAISS-Index die k ähnlichsten Chunks
3. Verschmilzt diese Chunks zu einem zusammenhängenden Kontextstring.

### Input
- prompt: str – z. B. eine Frage wie „Wie sieht ein Razepato aus?“
- k: int – Anzahl der gewünschten Treffer (Standard: 3).
- Benötigt global:
    - embed_model – das Sentence-Transformers-Modell.
    - index – FAISS-Index.
    - corpus – Liste der Chunks.

### Output
- retrieved_text: str – zusammengesetzter Text aus den k Top-Chunks, verbunden durch Trennlinie:
    - "\n\n---\n\n".join(retrieved_chunks)
- Optional (aktuell auskommentiert):
    - Debug-Ausgaben zu Indizes, Distanzen und einzelnen Chunks.

In [18]:
def get_rag_context(prompt, k=5):
    # Query-Embedding
    query_emb = embed_model.encode([prompt], convert_to_numpy=True)

    distances, indices = index.search(query_emb, k)

    #print("Treffer-Indizes:", indices, "Distanzen:", distances)

    #for rank, idx in enumerate(indices[0]):
    #    print(f"Rank {rank} – Distanz: {distances[0][rank]:.4f}")
    #    print("Chunk:")
    #    print(corpus[idx][:300])
    #    print("---")

     # alle k Treffer aus dem Corpus holen
    retrieved_chunks = [corpus[i] for i in indices[0]]

    # zu einem Kontext-String zusammenbauen
    retrieved_text = "\n\n---\n\n".join(retrieved_chunks)
    #print(retrieved_text[:800])

    return retrieved_text

## Nur Retrieval, noch kein LLM

### Beispiele
- Beispiel 1: "Erzähl mir eine Geschichte über den Mathematiker Maximus Qvestus der III."
- Beispiel 2: "Welche zentrale mathematische Idee wird Maximus Qvestus dem III zugeschrieben, und wie wird sie beschrieben?"
- Beispiel 3: "Welche Hinweise gibt es in der Geschichte darauf, dass Maximus Qvestus dem III als exzentrisch galt?"
- Beispiel 4: "Wie endet die Geschichte von Maximus Qvestus dem III, und welche mögliche symbolische Bedeutung hat die letzte Botschaft an der Tafel?"

### Funktion
- Demonstriert, wie man ganz ohne LLM, nur mit Embeddings + Index, relevante Inhalte abruft.

### Input
- Verschiedene rag_prompt

### Output
- rag_context: str – Textauszüge aus corpus, die semantisch zur Frage in Verbindung stehen.

### Konsolenausgabe:
- print(rag_context) – zeigt den reinen Kontext (z. B. Beschreibung von Lebensraum/Region des Razepato).

In [19]:
rag_prompt = "Erzähl mir eine Geschichte über den Mathematiker Maximus Qvestus der III."

rag_context = get_rag_context(rag_prompt)
print(rag_context)

Maximus Qvestus der III.

---

Sein größtes Werk, die sogenannte „Theorie der wandernden Zahlen“, entstand aus einer einfachen Beobachtung: Zahlen, so meinte Qvestus, verhalten sich wie Reisende — sie ändern ihre Bedeutung je nachdem, in welchem System man sie betrachtet.

---

Kollegen hielten ihn zunächst für exzentrisch, doch seine Vorlesungen waren legendär; er zeichnete Gleichungen mit farbiger Kreide und behauptete, jede Farbe habe eine eigene mathematische Stimmung.

---

Als man Jahre später sein Arbeitszimmer öffnete, fand man lediglich eine Tafel mit einem einzigen Satz: „Die schwierigste Gleichung ist die, die uns selbst enthält.“ Manche sagen, er habe die Lösung gefunden — und sei ihr gefolgt.

---

Mit sechs Jahren soll er begonnen haben, die Pflastersteine der Straße nach symmetrischen Mustern zu zählen — nicht aus Zwang, sondern aus reiner Freude an der Ordnung, die sich unter dem scheinbaren Chaos verbarg.


In [20]:
rag_prompt = "Welche zentrale mathematische Idee wird Maximus Qvestus dem III zugeschrieben, und wie wird sie beschrieben?"

rag_context = get_rag_context(rag_prompt)
print(rag_context)

Maximus Qvestus der III.

---

Sein größtes Werk, die sogenannte „Theorie der wandernden Zahlen“, entstand aus einer einfachen Beobachtung: Zahlen, so meinte Qvestus, verhalten sich wie Reisende — sie ändern ihre Bedeutung je nachdem, in welchem System man sie betrachtet.

---

Kollegen hielten ihn zunächst für exzentrisch, doch seine Vorlesungen waren legendär; er zeichnete Gleichungen mit farbiger Kreide und behauptete, jede Farbe habe eine eigene mathematische Stimmung.

---

Mit sechs Jahren soll er begonnen haben, die Pflastersteine der Straße nach symmetrischen Mustern zu zählen — nicht aus Zwang, sondern aus reiner Freude an der Ordnung, die sich unter dem scheinbaren Chaos verbarg.

---

Als man Jahre später sein Arbeitszimmer öffnete, fand man lediglich eine Tafel mit einem einzigen Satz: „Die schwierigste Gleichung ist die, die uns selbst enthält.“ Manche sagen, er habe die Lösung gefunden — und sei ihr gefolgt.


In [21]:
rag_prompt = "Welche Hinweise gibt es in der Geschichte darauf, dass Maximus Qvestus dem III als exzentrisch galt?"

rag_context = get_rag_context(rag_prompt)
print(rag_context)

Maximus Qvestus der III.

---

Sein größtes Werk, die sogenannte „Theorie der wandernden Zahlen“, entstand aus einer einfachen Beobachtung: Zahlen, so meinte Qvestus, verhalten sich wie Reisende — sie ändern ihre Bedeutung je nachdem, in welchem System man sie betrachtet.

---

Mit sechs Jahren soll er begonnen haben, die Pflastersteine der Straße nach symmetrischen Mustern zu zählen — nicht aus Zwang, sondern aus reiner Freude an der Ordnung, die sich unter dem scheinbaren Chaos verbarg.

---

Im Alter zog er sich in ein abgelegenes Observatorium zurück, wo er bis zu seinem Verschwinden an einem Buch arbeitete, das nur den Titel „Über das Unendliche und warum es Geduld verlangt“ trug.

---

Kollegen hielten ihn zunächst für exzentrisch, doch seine Vorlesungen waren legendär; er zeichnete Gleichungen mit farbiger Kreide und behauptete, jede Farbe habe eine eigene mathematische Stimmung.


In [22]:
rag_prompt = "Wie endet die Geschichte von Maximus Qvestus dem III, und welche mögliche symbolische Bedeutung hat die letzte Botschaft an der Tafel?"

rag_context = get_rag_context(rag_prompt)
print(rag_context)

Maximus Qvestus der III.

---

Sein größtes Werk, die sogenannte „Theorie der wandernden Zahlen“, entstand aus einer einfachen Beobachtung: Zahlen, so meinte Qvestus, verhalten sich wie Reisende — sie ändern ihre Bedeutung je nachdem, in welchem System man sie betrachtet.

---

Mit sechs Jahren soll er begonnen haben, die Pflastersteine der Straße nach symmetrischen Mustern zu zählen — nicht aus Zwang, sondern aus reiner Freude an der Ordnung, die sich unter dem scheinbaren Chaos verbarg.

---

Im Alter zog er sich in ein abgelegenes Observatorium zurück, wo er bis zu seinem Verschwinden an einem Buch arbeitete, das nur den Titel „Über das Unendliche und warum es Geduld verlangt“ trug.

---

Als man Jahre später sein Arbeitszimmer öffnete, fand man lediglich eine Tafel mit einem einzigen Satz: „Die schwierigste Gleichung ist die, die uns selbst enthält.“ Manche sagen, er habe die Lösung gefunden — und sei ihr gefolgt.


## Chat-Prompt mit RAG-Kontext: build_chat_prompt_with_rag

### Funktion
- Erweitert die frühere build_chat_prompt-Logik um automatisches RAG:
    - Für jede neue User-Nachricht wird zunächst get_rag_context(user_prompt) aufgerufen.
        - In dieser Version wird nicht unterschieden, ob dies notwendig ist oder nicht.
        - Im Produktivsystem müsste man das anpassen.
        - Hierzu könnte man eine Mindestdistanz festlegen.
    - Der zurückgegebene Kontext wird als zusätzliche System-Nachricht in den Chat eingefügt:
        - Klar markiert als „Kontext aus Wissensdatenbank, nicht vom User“.
    - Danach wird wie zuvor das Llama-Chat-Template verwendet, um einen geeignet formatierten Prompt zu bauen.

### Input
- system_prompt: Optional[str] – globale Instruktionen, hier später der RAG-Systemprompt.
- user_prompt: str – aktuelle Benutzernachricht.
- history: Optional[List[Tuple[str, str]]] – Dialogverlauf (user_text, assistant_text).

### Output
- prompt: str – voll formatierter Chat-Prompt, der folgende Komponenten enthält:
    - Rolle des Assistenten (System-Prompt),
    - bisherigen Verlauf,
    - RAG-Kontext (falls vorhanden),
    - aktuelle User-Frage,
    - Assistant-Start-Marker für die Generierung.

Damit wird das Modell gezielt „grounded“: Es sieht den abgerufenen Kontext explizit und kann ihn in die Antwort einbauen – ein klassischer Mechanismus zur Reduktion von Halluzinationen in RAG-Systemen.

In [23]:
from typing import List, Optional, Tuple, Dict

def build_chat_prompt_with_rag(
    system_prompt: Optional[str],
    user_prompt: str,
    history: Optional[List[Tuple[str, str]]] = None,
) -> str:
    """
    history: Liste von (user_text, assistant_text) Paaren für vorherigen Dialog.
    """
    messages: List[Dict[str, str]] = []

    rag_context = get_rag_context(user_prompt)

    if system_prompt:
        messages.append(
            {"role": "system",
             "content": system_prompt
             }
        )

    if history:
        for user_msg, assistant_msg in history:
            messages.append(
                {"role": "user",
                 "content": user_msg
                 }
            )
            messages.append(
                {"role": "assistant",
                 "content": assistant_msg
                 }
            )

    if rag_context:
        messages.append(
            {"role": "system",
             "content": (
                            "Das folgende ist Kontext aus einer Wissensdatenbank. "
                            "Er ist nicht vom User. Nutze ihn nur, wenn er relevant ist:\n\n"
                            f"{rag_context}"
                        )
             }
        )

    # aktuelle User-Nachricht
    messages.append(
        {"role": "user",
         "content": user_prompt
         }
    )

    # Llama-3.1 hat ein chat_template im Tokenizer hinterlegt
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,            # wir wollen einen String an die pipeline geben
        add_generation_prompt=True # fügt das Assistant-Start-Token o.ä. hinzu
    )
    return prompt

## System-Prompt für den RAG-Mathematikassistenten: system_prompt_rag

### Funktion
- Definiert einen neuen, ausführlichen System-Prompt, der:
    - die Rolle als Mathematikassistenten festlegt,
    - die Nutzung von RAG Kontext beschreibt,
    - den Umgang mit Unsicherheit und Halluzinationen explizit regelt,
    - Ablehnungsfälle (Out-of-Scope, Echtzeitdaten etc.) standardisiert.

### Input
- Hardcodierter String system_prompt_rag mit folgenden Kernelementen:
    - Aufgaben:
        - Mathematik-Fragen beantworten.
        - Mit zusätzlichem Kontext arbeiten („Kontext (aus Retrieval)“).
    - Kontextnutzung:
        - Kontext ist eine zuverlässige Wissensquelle.
        - Fakten im Kontext dürfen und sollen verwendet werden.
    - Unsicherheit:
        - Keine Spekulation, keine erfundenen Fakten.
        - Keine mathematischen Aufgaben lösen, wenn dazu kein MCP-Tool zur Verfügung steht, um diese Aufgabe es exakt zu lösen.
        - Wenn etwas weder im Weltwissen noch im Kontext steht, soll das klar kommuniziert werden.
    - Einschränkungen:
        - Keine Beantwortung fachfremder Themen.
        - Ablehnung, wenn explizit nach mathematischen Dingen gefragt wird.
    - Standardisierte Ablehnungssätze (zwei vorgegebene Formulierungen).

### Output
- system_prompt_rag: str – wird später als system_prompt in build_chat_prompt_with_rag(...) genutzt.
- Effekte auf das Modellverhalten:
    - Schärferer Fokus auf Reisethemen.
    - Explizite Erlaubnis, den RAG-Kontext zu verwenden.
    - Reduktion von Halluzinationen, indem Spekulation verboten wird und Lücken benannt werden müssen.

In [24]:
system_prompt_rag = """
Du bist ein persönlicher Mathematikassistent.

Deine Aufgaben:
- Beantworte ausschließlich Wissensfragen rund um das Thema Mathematik.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Umgang mit Wissen und Unsicherheit:
- Erfinde keine Fakten und spekuliere nicht.
- Wenn eine Information weder in deinem Weltwissen noch im bereitgestellten Kontext vorkommt, weise darauf hin.
- Falls dir wirklich Informationen fehlen, formuliere eine normale, erklärende Antwort und biete ggf. an, was du stattdessen aus dem Kontext sagen kannst.

Einschränkungen (Ablehnungsfälle):
- Echtzeit- oder Trenddaten benötigen
- Löse niemals mathematischen Aufgaben (z.B: 1 + 1, 2 * 2, 3 - 3, 4 / 4, ...).
- Informationen von externen Plattformen oder Datenbanken benötigen
- nicht zum Mathematik-Kontext gehörende Theman

Verwende beim Ablehnen aufgrund fehlender verlässlicher Informationen:
„Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.“

Verwende bei Themen, die klar nicht zum Reise-Kontext gehören:
„Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.“
"""

## RAG-gestützte LLM-Antwort: „Ich möchte ein Razepato beobachten…“

### Funktion
- Zeigt den kompletten End-to-End-Flow:
    1. User-Frage zum Reisen,
    2. Retrieval von Razepato-Kontext über get_rag_context,
    3. Einbau des Kontexts in den Chat-Prompt,
    4. Antwortgenerierung über llama_chat.

### Input
- user_prompt = "Ich möchte ein Razepato beobachten. Wohin muss ich reisen?"
- system_prompt = Vorher defineirter Systemprompt.
- history = Vorausgegangene Chats.

### Output
- Ein Antwort-String, in dem das LLM:
    - die Reisefrage beantwortet (z. B. wohin man reisen muss, um ein Razepato zu sehen),
    - idealerweise den RAG-Kontext nutzt (z. B. Lebensraum aus der Datei),
    - den System-Prompt beachtet (Reisekontext, keine Spekulation, evtl. Ablehnungssätze),
    - und weiterhin „Max Mustermann“ korrekt adressiert (wegen history).

Damit haben wir einen kompletten kleinen RAG-Stack gebaut:
Datei → Chunks → Embeddings → Index → Retriever → System-Prompt mit RAG-Kontext → LLM-Antwort.

Genau dieser Aufbau ist das Muster, das auch in größeren produktiven RAG-Systemen verwendet wird – nur mit mehr Daten, komplexerer Indizierung und oft zusätzlichen Guardrails.

In [25]:
user_prompt = "Erzähl mir eine Geschichte über den Mathematiker Maximus Qvestus der III."

history = ""

prompt = build_chat_prompt_with_rag(
    user_prompt=user_prompt,
    system_prompt=system_prompt_rag,
    history=history,
)

llama_chat(prompt, max_new_tokens=1024)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Ich kann dir eine Geschichte über Maximus Qvestus der III. erzählen, basierend auf dem bereitgestellten Kontext.\n\nEs war ein sonniger Nachmittag, als Maximus Qvestus der III. auf der Straße stand und die Pflastersteine zählte. Er war gerade sechs Jahre alt und hatte bereits eine Leidenschaft für die Mathematik entdeckt. Seine Augen leuchteten auf, als er die symmetrischen Muster entdeckte, die sich unter den Pflastersteinen verbargen.\n\nMit jedem Schritt, den er machte, zählte er die Steine und versuchte, die Muster zu verstehen. Seine Eltern sahen ihn lächelnd zu, als sie ihn dabei sahen, wie er sich in die Welt der Mathematik verlor.\n\nAls er älter wurde, entwickelte sich Maximus\' Leidenschaft für die Mathematik weiter. Er begann, Gleichungen zu zeichnen und behauptete, jede Farbe habe eine eigene mathematische Stimmung. Seine Vorlesungen waren legendär, und seine Studenten waren fasziniert von seiner Fähigkeit, komplexe Konzepte in einfache, farbenfrohe Bilder zu übersetzen.\n